In [57]:
import numpy as np
from interval import interval
np.set_printoptions(precision=2, suppress=True)

In [58]:
class Zonotope:
    def __init__(self, centre, generators):
        self.centre = np.array(centre).reshape(-1)         # shape (n,)
        self.generators = np.atleast_2d(generators)        # shape (n, m)
        if self.generators.shape[0] != self.centre.shape[0]:
            raise ValueError("Generators must have same dimension as centre")
        self.d = self.centre.shape[0]   # dimension
        self.m = self.generators.shape[1]  # number of generators

    def minkowski_sum(self, other):
        new_centre = self.centre + other.centre
        new_generators = np.hstack((self.generators, other.generators))
        return Zonotope(new_centre, new_generators)
    
    def subtract(self, other):
        new_centre = self.centre - other.centre
        new_generators = np.hstack((self.generators, -other.generators))
        return Zonotope(new_centre, new_generators)
    
    @classmethod
    def from_intervals(cls, lower, upper):
        lower = np.array(lower).reshape(-1)
        upper = np.array(upper).reshape(-1)

        if lower.shape != upper.shape:
            raise ValueError("Lower and upper must have same shape")

        centre = (lower + upper) / 2
        radii = (upper - lower) / 2

        generators = np.diag(radii)
        return cls(centre, generators)
    
    def output_interval(self):
        radius = np.sum(np.abs(self.generators), axis=1)  # sum across generators
        lower = self.centre - radius
        upper = self.centre + radius
        return lower, upper

    def affine_map(self, A, b=None):
            # Ensure centre is a column vector for matrix multiplication
            new_centre = A @ self.centre
            if b is not None:
                new_centre = new_centre + np.array(b).reshape(-1)
            new_generators = A @ self.generators
            return Zonotope(new_centre, new_generators)
    
    def __repr__(self):
        return f"Zonotope(dim={self.d}, generators={self.m})\nCentre:\n{self.centre}\nGenerators:\n{self.generators}"
    

def naive_bounds(X, y_lower, y_upper, x_grid=None):
    M = np.linalg.inv(X.T @ X) @ X.T 
    y_intervals = [interval[y_l, y_u] for y_l, y_u in zip(y_lower, y_upper)]

    beta_naive = []
    for i in range(M.shape[0]):
        row = M[i]
        beta_j = interval[0, 0]
        for c, y_int in zip(row, y_intervals):
            beta_j += float(c) * y_int
        beta_naive.append(beta_j)

    if x_grid is not None:
        beta0, beta1 = beta_naive
        y_min = []
        y_max = []
        for xg in x_grid:
            y_int = beta0 + beta1 * xg
            y_min.append(y_int[0][0])
            y_max.append(y_int[0][1])
        return beta_naive, (np.array(y_min), np.array(y_max))
    
    return beta_naive

### 1. Initial Zonotope

In [59]:
y1 = [-2.0, 3.0]
y2 = [2.0, 4.0]
y3 = [-5.0, -2.0]

lower = [y1[0], y2[0], y3[0]]
upper = [y1[1], y2[1], y3[1]]

X = np.array([[1, -1], 
              [1, 2],
              [1, 3]])


Z_y = Zonotope.from_intervals(lower, upper)
print(Z_y)


Zonotope(dim=3, generators=3)
Centre:
[ 0.5  3.  -3.5]
Generators:
[[2.5 0.  0. ]
 [0.  1.  0. ]
 [0.  0.  1.5]]


### 2. First Affine Transformation

In [60]:
M = np.linalg.inv(X.T @ X) @ X.T
print(M)

Z_beta = Z_y.affine_map(M)
print(Z_beta)

[[ 0.69  0.23  0.08]
 [-0.27  0.08  0.19]]
Zonotope(dim=2, generators=3)
Centre:
[ 0.77 -0.58]
Generators:
[[ 1.73  0.23  0.12]
 [-0.67  0.08  0.29]]


### 3. Second Affine Transformation

In [61]:
X_g = np.array([[1, 0],
                [1, 1],
                [1, 3]])

Z_yhat = Z_beta.affine_map(X_g)
print(Z_yhat)

Zonotope(dim=3, generators=3)
Centre:
[ 0.77  0.19 -0.96]
Generators:
[[ 1.73  0.23  0.12]
 [ 1.06  0.31  0.4 ]
 [-0.29  0.46  0.98]]


### 4. Output Intervals


In [ ]:
Z_yhat_lower, Z_yhat_upper = Z_yhat.output_interval()

print("Zonotope Regression Intervals:")
print(f'y_x1 : [{Z_yhat_lower[0]:.2f}, {Z_yhat_upper[0]:.2f}]')
print(f'y_x2 : [{Z_yhat_lower[1]:.2f}, {Z_yhat_upper[1]:.2f}]')
print(f'y_x3 : [{Z_yhat_lower[2]:.2f}, {Z_yhat_upper[2]:.2f}]')

print('\nNaive Intervals:')
_, (n_lower, n_upper) = naive_bounds(X, lower, upper, X_g)
print(f'y_x1 : [{n_lower[0]:.2f}, {n_upper[0]:.2f}]')
print(f'y_x2 : [{n_lower[1]:.2f}, {n_upper[1]:.2f}]')
print(f'y_x3 : [{n_lower[2]:.2f}, {n_upper[2]:.2f}]')

y_x1 : [-1.31, 2.85]
y_x2 : [-1.58, 1.96]
y_x3 : [-2.69, 0.77]

Naive Intervals:
y_x1 : [-2.92, 2.85]
y_x2 : [-2.92, 3.31]
y_x3 : [-2.92, 4.23]
